In [3]:
# import all necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import git

In [4]:
### Define file paths
base_root = git.Repo('.', search_parent_directories=True).working_tree_dir
base_input = base_root + '\\Input & Output\\Input\\'
base_output = base_root + '\\Input & Output\\Output\\'

supply_input = base_input + 'Supply\\'
supply_output = base_output + 'Supply\\'

supply_canada_stats = supply_input + 'Canada Stats\\'

In [5]:
### Combine supply data from Canada Statistics
# archive data, up to 2007-12
df_supply1 = pd.read_csv(supply_canada_stats + '2002 05 - 2007 12.csv')
df_supply1 = df_supply1[['REF_DATE', 'Electric power, components', 'VALUE']]
df_supply1 = df_supply1.rename(columns={'REF_DATE': 'Date', 'Electric power, components': 'Type of electricity generation', 'VALUE': 'MW Amount'})
df_supply1 = df_supply1.loc[df_supply1['Type of electricity generation'] == 'Overall total generation']

# present data, up to 2025-04
df_supply2 = pd.read_csv(supply_canada_stats + '2008 01 - 2025 04.csv')
df_supply2 = df_supply2[['REF_DATE', 'Type of electricity generation', 'VALUE']]
df_supply2 = df_supply2.rename(columns={'REF_DATE': 'Date', 'VALUE': 'MW Amount'})
df_supply2 = df_supply2.loc[df_supply2['Type of electricity generation'] == 'Total all types of electricity generation']

# combine archive and present data
df_supply = pd.concat([df_supply1, df_supply2])
df_supply = df_supply.drop('Type of electricity generation', axis = 1)
df_supply = df_supply.rename(columns={'REF_DATE': 'Date'})

# convert unit, MWh -> MW
df_supply['MW Amount'] = df_supply.apply(lambda row: row['MW Amount'] / pd.Timestamp(row['Date']).days_in_month / 24, axis=1)

# Grabing data until 2010
df_supply = df_supply.loc[pd.to_datetime(df_supply['Date']).dt.year < 2010]

# Expand to daily formate
df_supply['Date'] = pd.to_datetime(df_supply['Date'], format='%Y-%m')
expanded_rows = []
for _, row in df_supply.iterrows():
    start = row['Date']
    end = (start + pd.offsets.MonthEnd(1)).replace(hour=23)
    
    # Generate hourly datetime range for the month
    hourly_range = pd.date_range(start=start, end=end, freq='h')
    
    # Create hourly rows
    for hour in hourly_range:
        expanded_rows.append({'Date': hour, 'MW Amount': row['MW Amount']})

# Create the exploded DataFrame
df_canada = pd.DataFrame(expanded_rows)

In [ ]:
### Combine supply data from IESO Archive
supply_prefix = 'GOC-'
start_year = 2010
end_year = 2014

# Merging Dataframes
df_goc_supply_lst = []
for year in range(start_year, end_year + 1):
    df_goc_supply = pd.read_excel(supply_input + 'GOC\\' + supply_prefix + str(year) + '.xlsx', sheet_name = 'Output')
    df_goc_supply = df_goc_supply.rename(columns = {
    'DATE': 'Date',
    'HOUR': 'Hour'
    })
    df_goc_supply_lst.append(df_goc_supply)
    
df_goc = pd.concat(df_goc_supply_lst)

# Combine hourly data
generator_list = df_goc.columns.to_list()
generator_list.remove('Date')
generator_list.remove('Hour')
df_goc['MW Amount'] = df_goc[generator_list].sum(axis=1, numeric_only=True)

# Rename column names
df_goc = df_goc[['Date', 'Hour', 'MW Amount']]

# Combining date with hour
df_goc['Date'] = pd.to_datetime(df_goc['Date'])
df_goc['Date'] = df_goc['Date'] + pd.to_timedelta(df_goc['Hour'], unit='h')
df_goc = df_goc.drop('Hour', axis = 1)

C:\Users\CGOD\AppData\Local\Temp\ipykernel_44156\1207383819.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_goc['MW Amount'] = df_goc[generator_list].sum(axis=1, numeric_only=True)


In [7]:
### Combine supply data from IESO
supply_prefix = 'PUB_GenOutputbyFuelHourly_'
start_year = 2015
cur_year = 2025

# Merging Dataframes
df_pub_supply_lst = []
for year in range(start_year, cur_year + 1):
    df_pub_supply = pd.read_excel(supply_input + 'PUB_GenOutputbyFuelHourly\\' + supply_prefix + str(year) + '.xlsx', usecols = [6, 7, 10])
    df_pub_supply_lst.append(df_pub_supply)
    
df_pub = pd.concat(df_pub_supply_lst)

# Renaming Dataframe
df_pub = df_pub.rename(columns = {
    'ns1:Day': 'Date',
    'ns1:Hour': 'Hour',
    'ns1:Output': 'MW Amount'
})

# Combine hourly data
df_pub = df_pub.groupby(['Date', 'Hour'])['MW Amount'].sum().reset_index()

# Combining date with hour
df_pub['Date'] = pd.to_datetime(df_pub['Date'])
df_pub['Date'] = df_pub['Date'] + pd.to_timedelta(df_pub['Hour'], unit='h')
df_pub = df_pub.drop('Hour', axis = 1)

In [8]:
# Combine all supply dataframe into one
df_supply = pd.concat([df_canada, df_goc, df_pub])

In [9]:
### Process the combined dataframe
# daily average
df_supply_daily = df_supply.groupby(pd.Grouper(key='Date', freq='D'))['MW Amount'].sum().reset_index()
df_supply_daily['Date'] = df_supply_daily['Date'].dt.strftime('%Y-%m-%d')
df_supply_daily['MW Amount'] = df_supply_daily.apply(lambda row: row['MW Amount'] / 24, axis=1)

# monthly average
df_supply_monthly = df_supply.groupby(pd.Grouper(key='Date', freq='ME'))['MW Amount'].sum().reset_index()
df_supply_monthly['Date'] = df_supply_monthly['Date'].dt.strftime('%Y-%m')
df_supply_monthly['MW Amount'] = df_supply_monthly.apply(lambda row: row['MW Amount'] / pd.Timestamp(row['Date']).days_in_month / 24, axis=1)


In [10]:
### Outputting dataframes
with pd.ExcelWriter(supply_output + 'ontario_supply_time_series_data.xlsx') as writer:
    df_supply.to_excel(writer, sheet_name='cleaned raw data', index = False)
    
    # By month and day
    df_supply_daily.to_excel(writer, sheet_name='daily average', index = False)
    df_supply_monthly.to_excel(writer, sheet_name='monthly average', index = False)